In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import random

IMG_SIZE = 128

# Categories and labels
categories = ["Healthy", "Tumor"]
label_map = {"Healthy": 0, "Tumor": 1}

# Dataset paths
ct_path = "/content/drive/MyDrive/Dataset/Brain Tumor CT scan Images"
mri_path = "/content/drive/MyDrive/Dataset/Brain Tumor MRI images"


In [ ]:
def load_images_from_folder(folder, label, limit=None):
    data = []
    count = 0
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # grayscale
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append((img, label))
            count += 1
            if limit and count >= limit:  # optional limit
                break
    return data

data = []
modalities = {"CT": ct_path, "MRI": mri_path}

for modality, path in modalities.items():
    for category in categories:
        folder = os.path.join(path, category)
        label = label_map[category]
        data += load_images_from_folder(folder, label)

print("✅ Total samples loaded:", len(data))


✅ Total samples loaded: 9622


In [ ]:
# Shuffle dataset
random.shuffle(data)

# Convert to NumPy arrays
X = np.array([i[0] for i in data], dtype=np.float32).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array([i[1] for i in data], dtype=np.uint8)

# Normalize images
X = X / 255.0

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (9622, 128, 128, 1)
y shape: (9622,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set:", X_train.shape, y_train.shape)
print("Testing set:", X_test.shape, y_test.shape)



Training set: (7697, 128, 128, 1) (7697,)
Testing set: (1925, 128, 128, 1) (1925,)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Fit augmentation only on training data
datagen.fit(X_train)

print("✅ Preprocessing complete! Data ready for Milestone 2 (model training).")


✅ Preprocessing complete! Data ready for Milestone 2 (model training).


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),   # prevent overfitting
    layers.Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 28800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,843,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,848,129 (7.05 MB)

 Trainable params: 1,848,129 (7.05 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),  # augmented training data
    epochs=10,
    validation_data=(X_test, y_test),
    verbose=1
)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


241/241 ━━━━━━━━━━━━━━━━━━━━ 107s 438ms/step - accuracy: 0.6483 - loss: 0.6347 - val_accuracy: 0.8145 - val_loss: 0.4293
Epoch 2/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 102s 422ms/step - accuracy: 0.7868 - loss: 0.4700 - val_accuracy: 0.7179 - val_loss: 0.5336
Epoch 3/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 98s 408ms/step - accuracy: 0.8141 - loss: 0.4233 - val_accuracy: 0.8005 - val_loss: 0.4552
Epoch 4/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 114s 474ms/step - accuracy: 0.8162 - loss: 0.4119 - val_accuracy: 0.8358 - val_loss: 0.3910
Epoch 5/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 107s 445ms/step - accuracy: 0.8438 - loss: 0.3640 - val_accuracy: 0.8151 - val_loss: 0.4245
Epoch 6/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 108s 450ms/step - accuracy: 0.8459 - loss: 0.3575 - val_accuracy: 0.8577 - val_loss: 0.3614
Epoch 7/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 142s 450ms/step - accuracy: 0.8548 - loss: 0.3496 - val_accuracy: 0.7694 - val_loss: 0.5171
Epoch 8/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 104s 434ms/step - accuracy: 0.8663 - loss: 0.323

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"✅ Test Accuracy:{ test_acc*100:.2f}%")


61/61 - 8s - 125ms/step - accuracy: 0.8805 - loss: 0.3282
✅ Test Accuracy:88.05%


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = y_test
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int).flatten()

print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=["Healthy","Tumor"]))

61/61 ━━━━━━━━━━━━━━━━━━━━ 7s 114ms/step
Confusion Matrix:
 [[783  78]
 [152 912]]

Classification Report:
               precision    recall  f1-score   support

     Healthy       0.84      0.91      0.87       861
       Tumor       0.92      0.86      0.89      1064

    accuracy                           0.88      1925
   macro avg       0.88      0.88      0.88      1925
weighted avg       0.88      0.88      0.88      1925



In [ ]:
model.save("brain_tumor_cnn.h5")
print("✅ Model saved as brain_tumor_cnn.h5")


✅ Model saved as brain_tumor_cnn.h5
